In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_curve, auc, \
   precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score, matthews_corrcoef

In [ ]:
data = pd.read_csv(r"C:\Users\Nilanjan\Desktop\paper\Revision\v1\SHP2_inhibitory_activity_prediction\10_fold_cross_validation\train_10folds_208.csv")
data = data.iloc[:,1:]
data

In [ ]:
data['TARGET'].value_counts()

In [ ]:
data = data.drop(["IC50(microM)"],axis=1)

In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.iloc[:,:-1].copy()
    labels = df.pop('TARGET')
    df = {key: value[:,tf.newaxis] for key, value in dataframe.iloc[:,:-1].items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(data))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [ ]:
def get_normalization_layer(name, dataset):
    # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)
    
    # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [ ]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    
    # create a layer that turns integer values into integer indices.
    index = layers.IntegerLookup(max_tokens=max_tokens)
    
    # Prepare a `tf.data.Dataset` that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # Learn the set of possible values and assign them a fixed integer index.
    index.adapt(feature_ds)
    
    # Encode the integer indices.
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # Apply multi-hot encoding to the indices. The lambda function captures the
    # layer, so you can use them, or include them in the Keras Functional model later.
    
    return lambda feature: encoder(index(feature))

In [ ]:
def train_val_split(data, fold, batch_size):
    train = data[data.Kfold != fold].reset_index(drop=True)
    valid = data[data.Kfold == fold].reset_index(drop=True)
    train_ds = df_to_dataset(train, batch_size=batch_size)
    val_ds = df_to_dataset(valid, shuffle=False, batch_size=batch_size)
    
    return train_ds, val_ds, train, valid

In [ ]:
def preprocess_dataset(numerical_features, categorical_features):
    all_inputs = []
    encoded_features = []

    # Numerical features.
    for header in numerical_features:
        numeric_col = tf.keras.Input(shape=(1,), name=header)
        normalization_layer = get_normalization_layer(header, train_ds)
        encoded_numeric_col = normalization_layer(numeric_col)
        all_inputs.append(numeric_col)
        encoded_features.append(encoded_numeric_col)
    
    hacc_col = tf.keras.Input(shape=(1,), name='NumHAcceptors', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='NumHAcceptors',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=20)
    encoded_hacc_col = encoding_layer(hacc_col)

    all_inputs.append(hacc_col)
    encoded_features.append(encoded_hacc_col)
    
    scc_col = tf.keras.Input(shape=(1,), name='NumSaturatedCarbocycles', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='NumSaturatedCarbocycles',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=5)
    encoded_scc_col = encoding_layer(scc_col)

    all_inputs.append(scc_col)
    encoded_features.append(encoded_scc_col)
    
    fbc_col = tf.keras.Input(shape=(1,), name='fr_bicyclic', dtype='int64')

    encoding_layer = get_category_encoding_layer(name='fr_bicyclic',
                                                 dataset=train_ds,
                                                 dtype='int64',
                                                 max_tokens=10)
    encoded_fbc_col = encoding_layer(fbc_col)
    all_inputs.append(fbc_col)
    encoded_features.append(encoded_fbc_col)
    
    return all_inputs, encoded_features

In [ ]:
numerical_features = ['MaxPartialCharge', 'FpDensityMorgan2', 'BCUT2D_CHGLO', 'BCUT2D_MRHI',
       'PEOE_VSA12', 'PEOE_VSA6', 'SMR_VSA3', 'SlogP_VSA3', 'SlogP_VSA8', 'EState_VSA6']

categorical_features = ['NumHAcceptors', 'NumSaturatedCarbocycles', 'fr_bicyclic']

In [ ]:
class config:
    hidden_layer_1 = 16
    hidden_layer_2 = 64
    hidden_layer_3 = 128
    hidden_layer_4 = 256
    hidden_layer_5 = 32
    hidden_layer_6 = 8
    initializer = tf.keras.initializers.HeNormal()
    lr = 0.001
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    dropout = 0.1
    batch_size = 16
    epochs = 100

In [ ]:
def AnnClassifier(all_inputs, encoded_features):
    all_features = tf.keras.layers.concatenate(encoded_features)
    x = tf.keras.layers.Dense(units=config.hidden_layer_1, activation="relu", 
                              kernel_initializer=config.initializer)(all_features)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_2, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_3, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_4, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_5, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_6, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(all_inputs, output)
    model.compile(optimizer = config.optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
def evaluate(fold, all_inputs, encoded_features, valid, val_ds):
    
    model = AnnClassifier(all_inputs, encoded_features)
    checkpoint = f".\\model_checkpoints\\fold_{fold}"
    model.load_weights(checkpoint)
    
    y_pred_proba = model.predict(val_ds)
    y_true = valid.TARGET.values
    auc = roc_auc_score(y_true, y_pred_proba)
    y_pred = (y_pred_proba > 0.5)
    
    accuracy = accuracy_score(y_true,y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC

In [ ]:
final_features = numerical_features + categorical_features + ["TARGET","Kfold"]
data = data[final_features]
data

In [ ]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []


for fold_ in range(10):
    train_ds, val_ds, train, valid = train_val_split(data, fold_, config.batch_size)
    all_inputs, encoded_features = preprocess_dataset(numerical_features, categorical_features)
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC = evaluate(fold_, all_inputs, encoded_features, valid, val_ds)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")

In [ ]:
fold_metrics = pd.DataFrame(columns=['Accuracy','AUC','Precision_1','Precision_0','Recall_1','Recall_0','F1score','Kappa','MCC'])
fold_metrics['Accuracy'] = np.array(accuracies)
fold_metrics['AUC'] = np.array(aucs)
fold_metrics['Precision_1'] = np.array(precisions_1)
fold_metrics['Precision_0'] = np.array(precisions_0)
fold_metrics['Recall_1'] = np.array(recalls_1)
fold_metrics['Recall_0'] = np.array(recalls_0)
fold_metrics['F1score'] = np.array(f1scores)
fold_metrics['Kappa'] = np.array(kappas)
fold_metrics['MCC'] = np.array(MCCs)
fold_metrics

In [ ]:
fold_metrics.loc[10,:] = [np.mean(np.array(accuracies)), np.mean(np.array(aucs)), np.mean(np.array(precisions_1)),
                               np.mean(np.array(precisions_0)), np.mean(np.array(recalls_1)), np.mean(np.array(recalls_0)),
                            np.mean(np.array(f1scores)), np.mean(np.array(kappas)), np.mean(np.array(MCCs))]

fold_metrics

In [ ]:
fold_metrics.loc[11,:] = [np.std(np.array(accuracies)), np.std(np.array(aucs)), np.std(np.array(precisions_1)),
                               np.std(np.array(precisions_0)), np.std(np.array(recalls_1)), np.std(np.array(recalls_0)),
                            np.std(np.array(f1scores)), np.std(np.array(kappas)), np.std(np.array(MCCs))]

fold_metrics

In [ ]:
fold_metrics.index = ['Fold_0','Fold_1','Fold_2','Fold_3','Fold_4','Fold_5','Fold_6','Fold_7','Fold_8','Fold_9','Mean','Std']
fold_metrics

# Bayesian optimization

In [ ]:
from hyperopt import hp
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
def AnnClassifier_(all_inputs, encoded_features,space):
    all_features = tf.keras.layers.concatenate(encoded_features)
    x = tf.keras.layers.Dense(units=config.hidden_layer_1, activation="relu", 
                              kernel_initializer=config.initializer)(all_features)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_2, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_3, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    #x = tf.keras.layers.Dropout(hparams['dropout'][0])(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_4, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_5, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    x = tf.keras.layers.Dense(units=config.hidden_layer_6, activation="relu",
                             kernel_initializer=config.initializer)(x)
    x = tf.keras.layers.BatchNormalization(axis=-1)(x)
    output = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(all_inputs, output)
    config.lr = space["learning_rate"]
    model.compile(optimizer = config.optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
class BayesianOPT:
    def __init__(self, data, space):
        self.space = space
        self.data = data
        self.trials = Trials()
    
    def _objective(self,space):
        model = AnnClassifier_(all_inputs, encoded_features, space)
        accuracies = []
        for fold_ in range(10):
            train_ds, val_ds, train, valid = train_val_split(data, fold_, config.batch_size)
            
            # evaluation = [( X_train, y_train), ( X_test, y_test)]
        

            checkpoint = f".\\model_checkpoints\\fold_{fold_}"
            model.load_weights(checkpoint)


            pred = model.predict(val_ds)
            y_test = valid.TARGET.values
            accuracy = accuracy_score(y_test, pred>0.5)
            accuracies.append(accuracy)
        
        final_accuracy = np.mean(np.array(accuracies))
        #print ("SCORE:", final_accuracy)
        return {'loss': -final_accuracy, 'status': STATUS_OK }
    
    def search_hyperparameters(self):

        best_hyperparams = fmin(fn = self._objective,
                                space = self.space,
                                algo = tpe.suggest,
                                max_evals = 500,
                                trials = self.trials)
        return best_hyperparams

In [ ]:
space = {
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.1)),
    'epochs': hp.choice('epochs', [10, 20, 30, 40, 50]),
    'batch_size': hp.choice('batch_size', [16, 32, 64, 128]),
}

In [ ]:
bayes_opt = BayesianOPT(data, space)
best_params = bayes_opt.search_hyperparameters()
print(best_params)

In [ ]:
best_params["epochs"] = 10
best_params["batch_size"] = 16

# Train the model with optimized hyperparameters

In [ ]:
def train(fold, all_inputs, encoded_features, train_ds, val_ds, epochs):
    
    print(f"Fold : {fold}")
    early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    
    logdir = f'.\\tensorboard_logs\\scalars\\fold_{fold}\\'
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir,write_graph=True,update_freq=1)
    
    checkpoint_filepath = f'.\\model_checkpoints\\fold_{fold}\\'
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_filepath,
        monitor='val_accuracy',
        mode='max',
        save_best_only=True)
    
    model = AnnClassifier_(all_inputs, encoded_features, best_params)

    model.fit(train_ds, 
            epochs = epochs, 
            validation_data = val_ds,
            callbacks=[early_stop, tensorboard_callback, model_checkpoint_callback])
    
    # model_loss = pd.DataFrame(log.history.history)
    # model_loss.to_csv(f".\\Metrics_\\metrics_{fold}.csv", index=False)

In [ ]:
def my_lambda(x):
    return tf.cast(x, tf.float32)

In [ ]:
def run(fold, data, best_params):
    train_ds, val_ds, train_, valid = train_val_split(data, fold_, best_params["batch_size"])
    # initialize Logistic Regression model
    # train_ds = train_ds.map(my_lambda)
    # val_ds = val_ds.map(my_lambda)
    all_inputs, encoded_features = preprocess_dataset(numerical_features, categorical_features)
    train(fold, all_inputs, encoded_features, train_ds, val_ds, best_params["epochs"])
    model = AnnClassifier_(all_inputs, encoded_features, best_params)
    checkpoint = f".\\model_checkpoints\\fold_{fold}"
    model.load_weights(checkpoint)
    y_pred = model.predict(val_ds)
    y_pred = y_pred > 0.5
    y_true = valid.TARGET.values
    accuracy = accuracy_score(y_true,y_pred)
    auc = roc_auc_score(y_true, y_pred)
    precision_1 = precision_score(y_true,y_pred,pos_label=1)
    precision_0 = precision_score(y_true,y_pred,pos_label=0)
    recall_1 = recall_score(y_true,y_pred,pos_label=1)
    recall_0 = recall_score(y_true,y_pred,pos_label=0)
    f1score = f1_score(y_true,y_pred)
    kappa = cohen_kappa_score(y_true,y_pred)
    MCC = matthews_corrcoef(y_true,y_pred)
    print(f"Fold = {fold}, AUC = {auc}, Accuracy = {accuracy}, \
          Precision_1 = {precision_1}, Precision_0 = {precision_0}\
          Recall_1 = {recall_1}, Recall_0 = {recall_0}, F1Score = {f1score}, kappa = {kappa}, MCC = {MCC}")
    
    return auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model

In [ ]:
aucs, accuracies, precisions_1, precisions_0, recalls_1, recalls_0, f1scores, kappas, MCCs = [], [], [], [], [], [], [], [], []

for fold_ in range(10):
    auc, accuracy, precision_1, precision_0, recall_1, recall_0, f1score, kappa, MCC, model = run(fold_, data, best_params)
    aucs.append(auc)
    accuracies.append(accuracy)
    precisions_1.append(precision_1)
    precisions_0.append(precision_0)
    recalls_1.append(recall_1)
    recalls_0.append(recall_0)
    f1scores.append(f1score)
    kappas.append(kappa)
    MCCs.append(MCC)
    
print("\n")
print(f"Mean Scores: AUC = {np.mean(np.array(aucs))}, \
      Accuracy = {np.mean(np.array(accuracies))}, \
      Precision_1 = {np.mean(np.array(precisions_1))}, Precision_0 = {np.mean(np.array(precisions_0))}\
      Recall_1 = {np.mean(np.array(recalls_1))}, Recall_0 = {np.mean(np.array(recalls_0))}\
      F1Score = {np.mean(np.array(f1scores))} \
      Kappa = {np.mean(np.array(kappas))} \
      MCC = {np.mean(np.array(MCCs))}")